<h1 style='text-align: center'>SQL Queries</h1>

## Getting Data From A SQL Database

### The Structure of a SQL Query

<img src='images/sql_statement.jpg'/>

#### GROUP BY

- Group columns by similar values
- SELECT COUNT(id), city from students GROUP BY city

#### HAVING

- Use to apply filter AFTER a `GROUP BY` based on aggregate criteria 
- `WHERE` is applied for conditions prior to the `GROUP BY`, `HAVING` is applied afterwards

For example, if we had a table of student names and the courses they were taking, we could ask a question such as which classes have 3 or more students with the name Matt?

Such a query would look something like this:

```SQL
SELECT
  class,
  COUNT(student_name) AS number_of_matts
FROM student_courses
WHERE student_name = "Matt"
GROUP BY 1
HAVING COUNT(student_name) >= 3;
```

In [16]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('tutorial.db')
c = conn.cursor()

In [2]:
columns = [x[0] for x in c.execute('select * from students').description]

In [3]:
columns

['name',
 'birthdate',
 'siblings',
 'birth_place',
 'years_in_nyc',
 'favorite_food']

### Questions
1. What are the names of all of the students?
2. Which student has the most siblings?
3. How many students are only children?
4. Which 3 students have lived in NYC the shortest amount of time?
5. How many students are native New Yorkers?
6. Do any two students have the same favorite food?


1. What are the names of all of the students.

In [4]:
c.execute('''
SELECT name
FROM students
''').fetchall()

[('Sean Abu Wilson',),
 ('David Miller',),
 ('Abhijeet Kamble',),
 ('Samantha Jackson',),
 ('Anmol Srivats',),
 ('Ran Tokman',),
 ('Amy Li',),
 ('Florencia Leoni',),
 ('Austin Krause',),
 ('Natalie Overchuk',),
 ('Akshay Sharma',),
 ('Mohamad Eldebek',),
 ('Adam Dick',),
 ('Menachi Korn',),
 ('Miguel Peña',),
 ('Atiar Rahman',),
 ('Fhel Dimaano',),
 ('Maks Pazuniak',),
 ('Alex Mitrani',),
 ('Yish Lim',),
 ('Nicole Roach',),
 ('Omer Hakim',)]

2. Which student has the most siblings?

> This is great place to use a subquery. Encourage students who are initially struggling with a question along the lines of "How could you select the largest number of siblings that anyone has in the group?" From there, you can further push students with a hint if needed: "How can you now make a selection using the result of this, [embedded as a subquery]?"

In [22]:
c.execute("""
SELECT name, siblings FROM students ORDER by siblings DESC Limit 4
""").fetchall()


[('Florencia Leoni', 4),
 ('Mohamad Eldebek', 4),
 ('Menachi Korn', 4),
 ('Miguel Peña', 4)]

In [23]:
pd.read_sql_query('''
SELECT 
    name, 
    siblings
FROM students
WHERE siblings = (SELECT MAX(siblings) FROM students);

''',conn)

,name,siblings
0,Florencia Leoni,4
1,Mohamad Eldebek,4
2,Menachi Korn,4
3,Miguel Peña,4


3. How many students are only children?

In [6]:
c.execute("""
SELECT count(*) FROM students WHERE siblings = 0
""").fetchone()

(3,)

4. Which 3 students have lived in NYC the shortest amount of time? (How long has each lived in NYC?)
    

In [24]:
c.execute('''
SELECT name, years_in_nyc FROM students 
ORDER by 2 Limit 3;
''').fetchall()

[('Anmol Srivats', 0.05), ('Natalie Overchuk', 0.1), ('Austin Krause', 0.17)]

5. How many students are native New Yorkers?

In [27]:
# c.execute('''
# SELECT name FROM students 
# WHERE birth_place = 'New York, NY'
# ''').fetchall()

c.execute('''
SELECT name, birth_place FROM students 
WHERE birth_place LIKE '%NY%';
''').fetchall()



[('David Miller', 'New York, NY'),
 ('Amy Li', 'New York, NY'),
 ('Akshay Sharma', 'New York, NY'),
 ('Adam Dick', 'New York, NY'),
 ('Alex Mitrani', 'New York, NY'),
 ('Nicole Roach', 'Brooklyn, NY')]

6. Do any two students have the same favorite food?

This problem employs the `Having` clause.  Be sure to review the difference between the where and having clause here. (Where filters apply before the group by clause and conditions following the having clause are filters applied after the group by on the resulting aggregate [statistics].) A useful example in doing so, could be to modify the question to something with an additional filtering criterion such as 'do any native new yorkers have the same favorite food?' This would force students to use a where clause prior to the group by to filter the results. Alternatively, see the question below for an alternative but related problem on favorite foods.

In [11]:
c.execute("""SELECT favorite_food, count(favorite_food)
FROM students
GROUP BY favorite_food
HAVING count(favorite_food) > 1
""").fetchall()

[('pizza', 2), ('steak', 2)]

In [35]:
#Which students have same favorite food
pd.read_sql_query('''
SELECT DISTINCT
    s1.name, 
    s2.name,
    favorite_food
    
FROM students s1
JOIN students s2 USING(favorite_food)

WHERE s1.name != s2.name

GROUP BY 3

''',conn)


,name,name,favorite_food
0,David Miller,Akshay Sharma,pizza
1,Florencia Leoni,Nicole Roach,steak


## More Questions

What are the favorite foods of this classroom?

In [10]:
c.execute("""
SELECT distinct favorite_food FROM students 
""").fetchall()

[('guacamole',),
 ('pizza',),
 ('Biriyani',),
 ('chocolate chip cookies',),
 ('Sushi',),
 ('cheesecake',),
 ('dumplings',),
 ('steak',),
 ('Everything Bagels',),
 ('pasta',),
 ('Tabouleh',),
 ('noodles',),
 ('falafel',),
 ('rice',),
 ('Reeses Puffs',),
 ('Kare Kare',),
 ('pho',),
 ('Fusilli Sorrentina',),
 ('burgers',),
 ('Avocado',)]

7. Which student was born closest to the cohort's graduation date?

In [19]:
pd.read_sql_query('''
SELECT 
    name, 
    birthdate, 
    birthdate - '12/06' as diff
FROM students
ORDER BY (birthdate - '12/06') DESC

''',conn)

,name,birthdate,diff
0,Amy Li,12/29,0
1,Maks Pazuniak,12/19,0
2,Alex Mitrani,12/06,0
3,Anmol Srivats,11/19,-1
4,Yish Lim,11/21,-1
5,Austin Krause,10/22,-2
6,Samantha Jackson,08/04,-4
7,Miguel Peña,08/14,-4
8,Fhel Dimaano,08/20,-4
9,David Miller,06/06,-6


In [43]:
pd.read_sql_query('''
SELECT 
    name, 
    birthdate,
    abs(julianday('2019-' || substr(birthdate,1,2) || '-' || substr(birthdate,4,2)) - julianday('2019-08-26')) as days
FROM students
ORDER BY 3 
LIMIT 1

''',conn)

,name,birthdate,days
0,Fhel Dimaano,08/20,6.0
